# 第三章 PyTorch基础：Tensor和Autograd

## 3.1 Tensor

Tensor，又名张量，读者可能对这个名词似曾相识，因它不仅在PyTorch中出现过，它也是Theano、TensorFlow、
Torch和MxNet中重要的数据结构。关于张量的本质不乏深度的剖析，但从工程角度来讲，可简单地认为它就是一个数组，且支持高效的科学计算。它可以是一个数（标量）、一维数组（向量）、二维数组（矩阵）和更高维的数组（高阶数据）。Tensor和Numpy的ndarrays类似，但PyTorch的tensor支持GPU加速。

本节将系统讲解tensor的使用，力求面面俱到，但不会涉及每个函数。对于更多函数及其用法，读者可通过在IPython/Notebook中使用函数名加`?`查看帮助文档，或查阅PyTorch官方文档[^1]。

[^1]: http://docs.pytorch.org

In [2]:
from __future__ import print_function
import torch as t
t.__version__

'0.4.1'

###  3.1.1 基础操作

学习过Numpy的读者会对本节内容感到非常熟悉，因tensor的接口有意设计成与Numpy类似，以方便用户使用。但不熟悉Numpy也没关系，本节内容并不要求先掌握Numpy。

从接口的角度来讲，对tensor的操作可分为两类：

1. `torch.function`，如`torch.save`等。
2. 另一类是`tensor.function`，如`tensor.view`等。

为方便使用，对tensor的大部分操作同时支持这两类接口，在本书中不做具体区分，如`torch.sum (torch.sum(a, b))`与`tensor.sum (a.sum(b))`功能等价。

而从存储的角度来讲，对tensor的操作又可分为两类：

1. 不会修改自身的数据，如 `a.add(b)`， 加法的结果会返回一个新的tensor。
2. 会修改自身的数据，如 `a.add_(b)`， 加法的结果仍存储在a中，a被修改了。

函数名以`_`结尾的都是inplace方式, 即会修改调用者自己的数据，在实际应用中需加以区分。

#### 创建Tensor

在PyTorch中新建tensor的方法有很多，具体如表3-1所示。

表3-1: 常见新建tensor的方法

|函数|功能|
|:---:|:---:|
|Tensor(\*sizes)|基础构造函数|
|tensor(data,)|类似np.array的构造函数|
|ones(\*sizes)|全1Tensor|
|zeros(\*sizes)|全0Tensor|
|eye(\*sizes)|对角线为1，其他为0|
|arange(s,e,step|从s到e，步长为step|
|linspace(s,e,steps)|从s到e，均匀切分成steps份|
|rand/randn(\*sizes)|均匀/标准分布|
|normal(mean,std)/uniform(from,to)|正态分布/均匀分布|
|randperm(m)|随机排列|

这些创建方法都可以在创建的时候指定数据类型dtype和存放device(cpu/gpu).


其中使用`Tensor`函数新建tensor是最复杂多变的方式，它既可以接收一个list，并根据list的数据新建tensor，也能根据指定的形状新建tensor，还能传入其他的tensor，下面举几个例子。

In [3]:
# 指定tensor的形状
a = t.Tensor(2, 3)
a # 数值取决于内存空间的状态，print时候可能overflow

tensor([[5.7808e-21, 9.5849e-43, 5.0120e-21],
        [9.5849e-43, 2.1629e+23, 2.1364e-07]])

In [20]:
# 用list的数据创建tensor
b = t.Tensor([[1,2,3],[4,5,6]])
b
b = t.Tensor([1，2])
b

SyntaxError: invalid character in identifier (<ipython-input-20-cce3f5734e7e>, line 4)

In [5]:
b.tolist() # 把tensor转为list

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

`tensor.size()`返回`torch.Size`对象，它是tuple的子类，但其使用方式与tuple略有区别

In [6]:
b_size = b.size()
b_size

torch.Size([2, 3])

In [7]:
b.numel() # b中元素总个数，2*3，等价于b.nelement()

6

In [8]:
# 创建一个和b形状一样的tensor
c = t.Tensor(b_size)
# 创建一个元素为2和3的tensor
d = t.Tensor((2, 3))
c, d

(tensor([[1.3014e-18, 9.5849e-43, 5.0122e-21],
         [9.5849e-43, 5.0120e-21, 9.5849e-43]]), tensor([2., 3.]))

除了`tensor.size()`，还可以利用`tensor.shape`直接查看tensor的形状，`tensor.shape`等价于`tensor.size()`

In [9]:
c.shape

torch.Size([2, 3])

需要注意的是，`t.Tensor(*sizes)`创建tensor时，系统不会马上分配空间，只是会计算剩余的内存是否足够使用，使用到tensor时才会分配，而其它操作都是在创建完tensor之后马上进行空间分配。其它常用的创建tensor的方法举例如下。

In [10]:
t.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [11]:
t.zeros(2, 3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [12]:
t.arange(1, 6, 2)

tensor([1, 3, 5])

In [13]:
t.linspace(1, 10, 3)

tensor([ 1.0000,  5.5000, 10.0000])

In [14]:
t.randn(2, 3, device=t.device('cpu'))

tensor([[ 2.2335, -0.8245,  1.1715],
        [-0.2348, -0.3592,  1.5256]])

In [15]:
t.randperm(5) # 长度为5的随机排列

tensor([4, 1, 3, 2, 0])

In [16]:
t.eye(2, 3, dtype=t.int) # 对角线为1, 不要求行列数一致

tensor([[1, 0, 0],
        [0, 1, 0]], dtype=torch.int32)

`torch.tensor`是在0.4版本新增加的一个新版本的创建tensor方法，使用的方法，和参数几乎和`np.array`完全一致

In [17]:
scalar = t.tensor(3.14159) 
print('scalar: %s, shape of sclar: %s' %(scalar, scalar.shape))

scalar: tensor(3.1416), shape of sclar: torch.Size([])


In [18]:
vector = t.tensor([1, 2])
print('vector: %s, shape of vector: %s' %(vector, vector.shape))

vector: tensor([1, 2]), shape of vector: torch.Size([2])


In [19]:
tensor = t.Tensor(1,2) # 注意和t.tensor([1, 2])的区别
tensor.shape

torch.Size([1, 2])

In [25]:
matrix = t.tensor([[0.1, 1.2], [2.2, 3.1], [4.9, 5.2]])
matrix,matrix.shape

(tensor([[0.1000, 1.2000],
         [2.2000, 3.1000],
         [4.9000, 5.2000]]), torch.Size([3, 2]))

In [26]:
t.tensor([[0.11111, 0.222222, 0.3333333]],
                     dtype=t.float64,
                     device=t.device('cpu'))

tensor([[0.1111, 0.2222, 0.3333]], dtype=torch.float64)

In [27]:
empty_tensor = t.tensor([])
empty_tensor.shape

torch.Size([0])

#### 常用Tensor操作

通过`tensor.view`方法可以调整tensor的形状，但必须保证调整前后元素总数一致。`view`不会修改自身的数据，返回的新tensor与源tensor共享内存，也即更改其中的一个，另外一个也会跟着改变。在实际应用中可能经常需要添加或减少某一维度，这时候`squeeze`和`unsqueeze`两个函数就派上用场了。

In [28]:
a

tensor([[5.7808e-21, 9.5849e-43, 5.0120e-21],
        [9.5849e-43, 2.1629e+23, 2.1364e-07]])

In [29]:
a = t.arange(0, 6)
a.view(2, 3)

tensor([[0, 1, 2],
        [3, 4, 5]])

In [30]:
b = a.view(-1, 3) # 当某一维为-1的时候，会自动计算它的大小
b.shape

torch.Size([2, 3])

In [31]:
b.unsqueeze(1) # 注意形状，在第1维（下标从0开始）上增加“１” 
#等价于 b[:,None]
b[:, None].shape

torch.Size([2, 1, 3])

In [37]:
b.unsqueeze(1)

tensor([[[0, 1, 2]],

        [[3, 4, 5]]])

In [38]:
b.unsqueeze(-2) # -2表示倒数第二个维度

tensor([[[0, 1, 2]],

        [[3, 4, 5]]])

In [39]:
c = b.view(1, 1, 1, 2, 3)
c.squeeze(0) # 压缩第0维的“１”

tensor([[[[0, 1, 2],
          [3, 4, 5]]]])

In [40]:
c.squeeze() # 把所有维度为“1”的压缩

tensor([[0, 1, 2],
        [3, 4, 5]])

In [41]:
a[1] = 100
b # a修改，b作为view之后的，也会跟着修改

tensor([[  0, 100,   2],
        [  3,   4,   5]])

`resize`是另一种可用来调整`size`的方法，但与`view`不同，它可以修改tensor的大小。如果新大小超过了原大小，会自动分配新的内存空间，而如果新大小小于原大小，则之前的数据依旧会被保存，看一个例子。

In [42]:
b.resize_(1, 3)
b

tensor([[  0, 100,   2]])

In [43]:
b.resize_(3, 3) # 旧的数据依旧保存着，多出的大小会分配新空间
b

tensor([[            0,           100,             2],
        [            3,             4,             5],
        [2937757630464, 2937757630464,   73015146900]])

#### 索引操作

Tensor支持与numpy.ndarray类似的索引操作，语法上也类似，下面通过一些例子，讲解常用的索引操作。如无特殊说明，索引出来的结果与原tensor共享内存，也即修改一个，另一个会跟着修改。

In [44]:
a = t.randn(3, 4)
a

tensor([[-0.7459,  1.3151, -1.3605, -0.0202],
        [ 1.4614,  0.1581, -0.3121, -0.8221],
        [-0.6188, -0.3137,  0.6630, -1.5831]])

In [45]:
a[0] # 第0行(下标从0开始)

tensor([-0.7459,  1.3151, -1.3605, -0.0202])

In [46]:
a[:, 0] # 第0列

tensor([-0.7459,  1.4614, -0.6188])

In [47]:
a[0][2] # 第0行第2个元素，等价于a[0, 2]

tensor(-1.3605)

In [48]:
a[0, -1] # 第0行最后一个元素

tensor(-0.0202)

In [49]:
a[:2] # 前两行

tensor([[-0.7459,  1.3151, -1.3605, -0.0202],
        [ 1.4614,  0.1581, -0.3121, -0.8221]])

In [50]:
a[:2, 0:2] # 前两行，第0,1列

tensor([[-0.7459,  1.3151],
        [ 1.4614,  0.1581]])

In [51]:
print(a[0:1, :2]) # 第0行，前两列 
print(a[0, :2]) # 注意两者的区别：形状不同

tensor([[-0.7459,  1.3151]])
tensor([-0.7459,  1.3151])


In [52]:
# None类似于np.newaxis, 为a新增了一个轴
# 等价于a.view(1, a.shape[0], a.shape[1])
a[None].shape

torch.Size([1, 3, 4])

In [53]:
a[None].shape # 等价于a[None,:,:]

torch.Size([1, 3, 4])

In [54]:
a[:,None,:].shape

torch.Size([3, 1, 4])

In [55]:
a[:,None,:,None,None].shape

torch.Size([3, 1, 4, 1, 1])

In [56]:
a > 1 # 返回一个ByteTensor

tensor([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0]], dtype=torch.uint8)

In [57]:
a[a>1] # 等价于a.masked_select(a>1)
# 选择结果与原tensor不共享内存空间

tensor([1.3151, 1.4614])

In [58]:
a[t.LongTensor([0,1])] # 第0行和第1行

tensor([[-0.7459,  1.3151, -1.3605, -0.0202],
        [ 1.4614,  0.1581, -0.3121, -0.8221]])

其它常用的选择函数如表3-2所示。

表3-2常用的选择函数

函数|功能|
:---:|:---:|
index_select(input, dim, index)|在指定维度dim上选取，比如选取某些行、某些列
masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor进行选取
non_zero(input)|非0元素的下标
gather(input, dim, index)|根据index，在dim维度上选取数据，输出的size与index一样


`gather`是一个比较复杂的操作，对一个2维tensor，输出的每个元素如下：

```python
out[i][j] = input[index[i][j]][j]  # dim=0
out[i][j] = input[i][index[i][j]]  # dim=1
```
三维tensor的`gather`操作同理，下面举几个例子。

In [60]:
a = t.arange(0, 16).view(4, 4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [61]:
# 选取对角线的元素
index = t.LongTensor([[0,1,2,3]])
a.gather(0, index)

tensor([[ 0,  5, 10, 15]])

In [62]:
index = t.LongTensor([[3,2,1,0]]).t()
a.gather(1, index)

tensor([[ 3],
        [ 6],
        [ 9],
        [12]])

In [63]:
# 选取反对角线上的元素，注意与上面的不同
index = t.LongTensor([[3,2,1,0]])
a.gather(0, index)

tensor([[12,  9,  6,  3]])

与`gather`相对应的逆操作是`scatter_`，`gather`把数据从input中按index取出，而`scatter_`是把取出的数据再放回去。注意`scatter_`函数是inplace操作。

```python
out = input.gather(dim, index)
-->近似逆操作
out = Tensor()
out.scatter_(dim, index)
```

In [65]:
# 把两个对角线元素放回去到指定位置
c = t.zeros(4,4)
c.scatter_(1, index, b)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.LongTensor for argument #4 'src'

对tensor的任何索引操作仍是一个tensor，想要获取标准的python对象数值，需要调用`tensor.item()`, 这个方法只对包含一个元素的tensor适用

In [66]:
a[0,0] #依旧是tensor）

tensor(0)

In [67]:
a[0,0].item() # python float

0

In [68]:
d = a[0:1, 0:1, None]
print(d.shape)
d.item() # 只包含一个元素的tensor即可调用tensor.item,与形状无关

torch.Size([1, 1, 1])


0

In [69]:
# a[0].item()  ->
# raise ValueError: only one element tensors can be converted to Python scalars

#### 高级索引
PyTorch在0.2版本中完善了索引操作，目前已经支持绝大多数numpy的高级索引[^10]。高级索引可以看成是普通索引操作的扩展，但是高级索引操作的结果一般不和原始的Tensor贡献内出。 
[^10]: https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing

In [70]:
x = t.arange(0,27).view(3,3,3)
x

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])

In [71]:
x[[1, 2], [1, 2], [2, 0]] # x[1,1,2]和x[2,2,0]

tensor([14, 24])

In [72]:
x[[2, 1, 0], [0], [1]] # x[2,0,1],x[1,0,1],x[0,0,1]

tensor([19, 10,  1])

In [73]:
x[[0, 2], ...] # x[0] 和 x[2]

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])